In [1]:
import sys
sys.path.append('..')
import cortex_etl as c_etl

ma = c_etl.analysis_initial_processing("../configs/5-FullCircuit/5-FullCircuit-2-BetterMinis-Fpr15-StimScan-10x.yaml", loglevel="ERROR")

In [2]:
a = ma.AllCompartments_spikes
a_name = "AllCompartments_spikes"
ma.analyzers[a_name]

In [3]:
# c_etl.post_analysis(a)
# c_etl.flatspace_videos(a)

In [5]:
import os
import numpy
import tqdm
def plot(Hs, t_bins, loc_bins, images_dir, delete_images, video_output_root, min_color_lim_pct=-1):
    if not os.path.isdir(images_dir):
        _ = os.makedirs(images_dir)
    from matplotlib import pyplot as plt
    # for bin_index in list(range(len(t_bins))):
    # flattened_Hs = Hs.flatten()
    # print(flattened_Hs.shape)
    # print(numpy.max(flattened_Hs))
    mx_clim = numpy.percentile(Hs, 99)
    mn_clim = 0
    if (min_color_lim_pct != -1):
        mn_clim = numpy.percentile(Hs, 20)
    # mx_clim = numpy.max(Hs)
    # mx_clim = numpy.percentile(Hs[Hs > 0], 90)
    fps = []
    for t_start, t_end, bin_index in tqdm.tqdm(zip(t_bins[:-1], t_bins[1:], list(range(len(t_bins))))):
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_axes([0.05, 0.05, 0.9, 0.9])
        img = ax.imshow(Hs[bin_index, :, :])
        img.set_clim([mn_clim, mx_clim])
        ax.set_title("{0} - {1} ms".format(t_start, t_end))
        plt.colorbar(img, cmap='plasma', label='FR (spikes / s')
        plt.box(False)
        fn = "frame{:04d}.png".format(bin_index)
        fp = os.path.join(images_dir, fn)
        fig.savefig(fp)
        fps.append(fp)
        plt.close(fig)

#     print(video_output_root + ".mp4")
    c_etl.video_from_image_files(fps, video_output_root + ".mp4")
    if delete_images:
        for f in fps:
            os.remove(f)

import numpy
from scipy.ndimage import gaussian_filter
def single_flatspace_video(simulation_row, filtered_dataframes, flat_locations, flatspace_video_opt, analysis_config):

    window_row = filtered_dataframes['windows'].iloc[0]

    images_dir = str(window_row['flatspace_video_images_dir']) + "/" + flatspace_video_opt['vid_str'] + "_" + str(simulation_row['simulation_id']) + "/"
    flatspace_path_pre = flatspace_video_opt['video_output_root'] + str(simulation_row['simulation_id']) + "_" + simulation_row['simulation_string']

    t_bins = c_etl.make_t_bins(window_row['t_start'], window_row['t_stop'], flatspace_video_opt['t_step'])
    spikes = filtered_dataframes['spikes'].loc[:, ['time', 'gid']].set_index('time')
#     gids = filtered_dataframes['neurons'].etl.q(circuit_id=simulation_row['circuit_id'])['gid']
#     locations = filtered_dataframes['circuits'].etl.q(circuit_id=simulation_row['circuit_id']).iloc[0]['circuit'].cells.get(gids, ["x", "y", "z"])
#     flat_locations = c_etl.flatten_locations(locations, analysis_config['flatmap'])
    
    loc_bins = c_etl.make_spatial_bins(flat_locations, flatspace_video_opt['n_spatial_bins'])
    spatial_temporal_hist = c_etl.make_histogram_function(t_bins, loc_bins, flat_locations, spikes)
    smoothed_spatial_temporal_hist = gaussian_filter(spatial_temporal_hist, [flatspace_video_opt['temporal_smoothing_sigma'], 1.0, 1.0])

    plot(smoothed_spatial_temporal_hist, t_bins, loc_bins, images_dir, flatspace_video_opt['delete_images'], flatspace_path_pre)

    hist = smoothed_spatial_temporal_hist
    hist_mean = numpy.mean(hist, axis=0)
    c_etl.plot_and_save_single_image(hist_mean, flatspace_path_pre + '_hist_mean.pdf')

    if (flatspace_video_opt['stim_anal'] != None):

        # where_stim = numpy.argwhere(numpy.logical_and(((t_bins) >= 20.0), ((t_bins) % 500.0) < 100.0)).flatten()
        # where_not_stim = numpy.argwhere(numpy.logical_and(((t_bins) >= 0.0), ((t_bins) % 500.0) >= 100.0)).flatten()

        where_stim = numpy.argwhere(numpy.logical_and(((t_bins) >= flatspace_video_opt['stim_anal']['stim_period'][0]), ((t_bins)) < flatspace_video_opt['stim_anal']['stim_period'][1])).flatten()
        where_not_stim = numpy.argwhere(numpy.logical_and(((t_bins) >= flatspace_video_opt['stim_anal']['spont_period'][0]), ((t_bins)) < flatspace_video_opt['stim_anal']['spont_period'][1])).flatten()

        hist_stim = hist[where_stim[:-1]]
        hist_not_stim = hist[where_not_stim[:-1]]
        hist_stim_mean = numpy.mean(hist_stim, axis=0)
        hist_not_stim_mean = numpy.mean(hist_not_stim, axis=0)

        hist_stim_mean_diff = hist_stim_mean - hist_not_stim_mean
        log_hist_stim_mean_diff = numpy.log(hist_stim_mean_diff)

        stim_minus_spont = hist_stim - hist_not_stim_mean

#         plot(stim_minus_spont, t_bins[where_stim], loc_bins, images_dir, flatspace_video_opt['delete_images'], flatspace_path_pre + '_stim_minus_spont')
        plot(stim_minus_spont, t_bins[where_stim], loc_bins, images_dir, flatspace_video_opt['delete_images'], flatspace_path_pre + '_stim_minus_spont_min_lim_99', min_color_lim_pct=99)
        plot(stim_minus_spont, t_bins[where_stim], loc_bins, images_dir, flatspace_video_opt['delete_images'], flatspace_path_pre + '_stim_minus_spont_min_lim_99_log', min_color_lim_pct=99)
#         plot(numpy.log(hist_stim - hist_not_stim_mean), t_bins[where_stim], loc_bins, images_dir, flatspace_video_opt['delete_images'], flatspace_path_pre + 'log_subtrac_mean')

        c_etl.plot_and_save_single_image(hist_not_stim_mean, flatspace_path_pre + '_hist_not_stim_mean.pdf')
        c_etl.plot_and_save_single_image(hist_stim_mean, flatspace_path_pre + '_hist_stim_mean.pdf')
        c_etl.plot_and_save_single_image(hist_stim_mean_diff, flatspace_path_pre + '_hist_stim_mean_diff.pdf')
        c_etl.plot_and_save_single_image(log_hist_stim_mean_diff, flatspace_path_pre + '_log_hist_stim_mean_diff.pdf')
        c_etl.plot_and_save_single_image(log_hist_stim_mean_diff, flatspace_path_pre + '_log_hist_stim_mean_diff_-4_-2.pdf')


    r_dict = {"smoothed_spatial_temporal_hist": smoothed_spatial_temporal_hist,
            "t_bins": t_bins}
    return r_dict


import os
from blueetl.parallel import call_by_simulation
from functools import partial
def flatspace_videos(a):

    print("\n----- Flatspace videos -----")
    for flatspace_video_key in a.analysis_config.custom['flatspace_videos']:
        flatspace_video_opt = a.analysis_config.custom['flatspace_videos'][flatspace_video_key]
        flatspace_video_opt['vid_str'] = flatspace_video_opt['window'] + "_" + str(flatspace_video_opt['t_step']) + "_" + str(flatspace_video_opt['n_spatial_bins']) + "_" + str(flatspace_video_opt['temporal_smoothing_sigma'])
        flatspace_video_opt['video_output_root'] = str(a.figpaths.flatspace_videos) + "/" + flatspace_video_opt['vid_str'] + "/"
        os.makedirs(flatspace_video_opt['video_output_root'], exist_ok=True)

        dataframes={
            "circuits": a.repo.simulations.df.loc[:, ['circuit', 'circuit_id', 'simulation_id']],
            "spikes": a.repo.report.df.etl.q(neuron_class="ALL", window=flatspace_video_opt['window']),
            "windows": a.repo.windows.df.etl.q(window=flatspace_video_opt['window']), 
            "neurons": a.repo.neurons.df.etl.q(neuron_class="ALL")}

        gids = a.repo.neurons.df.etl.q(circuit_id=0)['gid']
        locations = a.repo.simulations.df.loc[:, ['circuit', 'circuit_id', 'simulation_id']].iloc[0]['circuit'].cells.get(gids, ["x", "y", "z"])
        flat_locations = c_etl.flatten_locations(locations, a.analysis_config.custom["flatmap"])
        
        results = call_by_simulation(a.repo.simulations.df, 
                                        dataframes, 
                                        func=partial(single_flatspace_video, flat_locations=flat_locations, flatspace_video_opt=flatspace_video_opt, analysis_config=a.analysis_config.custom),
                                    how='series')



def flatspace_videos(a):

    print("\n----- Flatspace videos -----")
    for flatspace_video_key in a.analysis_config.custom['flatspace_videos']:
        flatspace_video_opt = a.analysis_config.custom['flatspace_videos'][flatspace_video_key]
        flatspace_video_opt['vid_str'] = flatspace_video_opt['window'] + "_" + str(flatspace_video_opt['t_step']) + "_" + str(flatspace_video_opt['n_spatial_bins']) + "_" + str(flatspace_video_opt['temporal_smoothing_sigma'])
        flatspace_video_opt['video_output_root'] = str(a.figpaths.flatspace_videos) + "/" + flatspace_video_opt['vid_str'] + "/"
        os.makedirs(flatspace_video_opt['video_output_root'], exist_ok=True)

        dataframes={
            "circuits": a.repo.simulations.df.loc[:, ['circuit', 'circuit_id', 'simulation_id']],
            "spikes": a.repo.report.df.etl.q(neuron_class="ALL", window=flatspace_video_opt['window']),
            "windows": a.repo.windows.df.etl.q(window=flatspace_video_opt['window']), 
            "neurons": a.repo.neurons.df.etl.q(neuron_class="ALL")}

        gids = a.repo.neurons.df.etl.q(circuit_id=0)['gid']
        locations = a.repo.simulations.df.loc[:, ['circuit', 'circuit_id', 'simulation_id']].iloc[0]['circuit'].cells.get(gids, ["x", "y", "z"])
        flat_locations = c_etl.flatten_locations(locations, a.analysis_config.custom["flatmap"])
        
        results = call_by_simulation(a.repo.simulations.df, 
                                        dataframes, 
                                        func=partial(single_flatspace_video, flat_locations=flat_locations, flatspace_video_opt=flatspace_video_opt, analysis_config=a.analysis_config.custom),
                                    how='series')
        
flatspace_videos(a)





----- Flatspace videos -----


100it [00:19,  5.02it/s]
100it [00:20,  5.00it/s]
100it [00:20,  4.96it/s]
/nvme/isbister/1502611/ipykernel_165043/1809533423.py:78: RuntimeWarning: divide by zero encountered in log
/nvme/isbister/1502611/ipykernel_165043/1809533423.py:78: RuntimeWarning: invalid value encountered in log
0it [00:00, ?it/s]/nvme/isbister/1502611/ipykernel_165043/1809533423.py:78: RuntimeWarning: divide by zero encountered in log
/nvme/isbister/1502611/ipykernel_165043/1809533423.py:78: RuntimeWarning: invalid value encountered in log
1it [00:00,  5.56it/s]/nvme/isbister/1502611/ipykernel_165043/1809533423.py:78: RuntimeWarning: divide by zero encountered in log
/nvme/isbister/1502611/ipykernel_165043/1809533423.py:78: RuntimeWarning: invalid value encountered in log
29it [00:05,  5.30it/s]
29it [00:05,  5.17it/s]
29it [00:05,  5.02it/s]
29it [00:05,  5.15it/s]
29it [00:05,  5.04it/s]
29it [00:05,  4.91it/s]
